In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
#import pyranges as pr
import matplotlib.pyplot as plt
from numpy import log10
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests
import statannot

import warnings
warnings.filterwarnings('ignore')

In [ ]:
file = './source_data/hubgene/hubgene_20.txt'
hub60= pd.read_csv(file, sep = '\t', header = 0)

In [ ]:
#file = './source_data/TCGA/TcgaTargetGtex_RSEM_Hugo_norm_count.txt'
#rsem_norm_count = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/TCGA/TcgaTargetGTEX_phenotype.txt'
phenotype = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/TCGA/TCGA_GTEX_BLOOD.txt'
rsem_norm_count = pd.read_csv(file, sep = '\t', header = 0)
rsem_norm_count

#file = '/home/chm/CANCER/TcgaTargetGtex_rsem_gene_tpm.txt'
#rsem_tpm = pd.read_csv(file, sep = '\t', header = 0)

rsem_norm_count_idx = rsem_norm_count.set_index('sample')

In [ ]:
# GBN = GTEX_Blood_Normal 
# GBE = GTEX_Blood_EBV
# TBA = TCGA_Blood_AML

GBN = phenotype[(phenotype['_study'] == 'GTEX')
                & phenotype['_primary_site'].str.contains('Blood') 
                & phenotype['_sample_type'].str.contains('Normal Tissue')
                & (phenotype['_primary_site'] =='Blood')]
GBN['Group'] = 'GBN'

GBE = phenotype[(phenotype['_study'] == 'GTEX')
                & phenotype['_primary_site'].str.contains('Blood') 
                & phenotype['_sample_type'].str.contains('Cell Line')
                & (phenotype['_primary_site'] =='Blood')]
GBE['Group'] = 'GBE'

TBA = phenotype[(phenotype['detailed_category'] == 'Acute Myeloid Leukemia')
                & (phenotype['_sample_type'] == 'Primary Blood Derived Cancer - Peripheral Blood')
                & (phenotype['_study'] == 'TCGA')]
TBA['Group'] = 'TBA'


print(GBN.shape)
print(GBE.shape)
print(TBA.shape)

In [ ]:
print(GBN[GBN['_gender'] == 'Male'].shape)
print(GBE[GBE['_gender'] == 'Male'].shape)
print(TBA[TBA['_gender'] == 'Male'].shape)

print(GBN[GBN['_gender'] == 'Female'].shape)
print(GBE[GBE['_gender'] == 'Female'].shape)
print(TBA[TBA['_gender'] == 'Female'].shape)

In [ ]:
GBNM = GBN[GBN['_gender'] == 'Male']
GBEM = GBE[GBE['_gender'] == 'Male']
TBAM = TBA[TBA['_gender'] == 'Male']

GBNF = GBN[GBN['_gender'] == 'Female']
GBEF = GBE[GBE['_gender'] == 'Female']
TBAF = TBA[TBA['_gender'] == 'Female']

# TGCA_plot

In [ ]:
import statannot
import scikit_posthocs as sp
import matplotlib.gridspec as gridspec

In [ ]:
#### cancer gene list
file = './source_data/annotation_files/cancer_gene_census.csv'
cancer_gene = pd.read_csv(file,encoding='latin-1')
cancer_gene['Role'] = cancer_gene['Role in Cancer'].astype(str)
cancer_gene['role_list'] = cancer_gene.Role.str.split(', ')
cancer_gene['oncogene'] = cancer_gene.apply(lambda x: 'Y' if 'oncogene' in x['role_list'] else 'N', axis=1)
cancer_gene['TSG'] = cancer_gene.apply(lambda x: 'Y' if 'TSG' in x['role_list'] else 'N', axis=1)
cancer_gene['fusion'] = cancer_gene.apply(lambda x: 'Y' if 'fusion' in x['role_list'] else 'N', axis=1)

cancer_gene = cancer_gene[['Gene Symbol','oncogene','TSG']]
cancer_gene = cancer_gene.rename(columns={'Gene Symbol':'gene'})

In [ ]:
def TCGA_plot(GENE_list,Tit):

    #for i, sex in [[[GBN,TBA],'all'],[[GBNM,TBAM],'male'],[[GBNF,TBAF],'female']]:
    for i, sex in [[[GBN,TBA],'all']]:
        
        N = len(GENE_list)

        fig = plt.figure(figsize = ((N/2+12),10*2))
        gs = gridspec.GridSpec(nrows=3*2, ncols=10+1, height_ratios=[1]*(3*2),width_ratios=[0.5]+[1]*10,hspace=0.5,wspace=0.4)    

        GT = pd.DataFrame()
        for n in range(len(i)):
            Inter_lst = list(set(i[n]['sample']) & set(rsem_norm_count_idx.columns))
            G = rsem_norm_count_idx[Inter_lst] 
            GT = pd.concat([GT,G],axis=1) 
            
        iN = 0
        jN = 1
        jON = 1
        
        
        RESULT = pd.DataFrame()
        for j in GENE_list:  

            GENE = GT[GT.index == j]
 
            if GENE.empty:
                continue
            else:
                GENE_T = GENE.transpose()
                GENE_T.columns = ['value']
                #print(GENE_T.shape)
                #print(GENE_T.head(10))
                GENE_T = GENE_T.reset_index(drop=False)
                #print(GENE_T.head(10))
    
                GENE_T['Group'] =  GENE_T.apply(lambda x : 'GBN' if x['index'] in GBN['sample'].unique().tolist()
                                                else 'GBE' if x['index'] in GBE['sample'].unique().tolist()
                                                else 'TBA', axis=1)
        
        
                NAME = 'PT_1' if iN == 0 or iN==1 else 'PT_2' if iN == 2 or iN == 3 else 'PT_4'
                COL = '#66c2a5' if iN == 0 or iN==1 else '#8da0cb' if iN == 2 or iN == 3 else '#deb887'
            
                plt.subplot(gs[iN,0])
                plt.xticks([])
                plt.yticks([])
                plt.gca().spines['right'].set_visible(False)
                plt.gca().spines['left'].set_visible(False)
                plt.gca().spines['top'].set_visible(False)
                plt.gca().spines['bottom'].set_visible(False)
                plt.text(0.7, 0.5, NAME, ha='center', va='center', size=25,  rotation=90, color=COL,fontweight='bold')
            
            
                plt.subplot(gs[iN,jN])
                PAL = ['pink' if i == 'TBA' else 'orange' if i == 'GBE' else 'skyblue' for i in GENE_T['Group'].unique()]
                ax = sns.stripplot(data=GENE_T,x='Group',y='value',jitter=True, alpha=0.5,palette=PAL,edgecolor=['black'],linewidth=0.5)
                ax = sns.boxplot(data=GENE_T,x='Group',y='value',showfliers=False,color='white', width=0.5)
            
                if len(GENE_T['Group'].unique()) == 3:
                    statannot.add_stat_annotation(ax, data = GENE_T, x='Group',y='value', test='Mann-Whitney',
                                                  box_pairs = [('GBN','GBE'),('GBN','TBA')], text_format='star',loc='inside')
                else :
                    statannot.add_stat_annotation(ax, data = GENE_T, x='Group',y='value', test='Mann-Whitney',
                                                  box_pairs = [set(GENE_T['Group'].unique())], text_format='star',loc='inside')
                
                plt.xlabel('')  
                plt.ylabel('')
                plt.xticks(fontsize=15)
                plt.yticks(fontsize=15)
            
                OS = cancer_gene.set_index('gene')
                OS_ = cancer_gene[cancer_gene['gene'] ==j]

                if OS_.empty:
                    fsize=25
                    color='black'
                    fweight='normal'
                
                elif OS.loc[j,'oncogene'] == 'Y' and OS.loc[j,'TSG'] == 'Y':
                    fsize=25
                    color='green'
                    fweight='bold'

                elif OS.loc[j,'oncogene'] == 'Y' and OS.loc[j,'TSG'] == 'N':
                    fsize=25
                    color='red'
                    fweight='bold'
                
                elif OS.loc[j,'oncogene'] == 'N' and OS.loc[j,'TSG'] == 'Y':
                    fsize=25
                    color='blue'
                    fweight='bold'
                
                else:
                    fsize=25
                    color='black'
                    fweight='normal'
                
                fontdict={'fontname': 'Times New Roman',
                          'fontsize': fsize,
                          'color' : color,
                          'style': 'oblique',
                          'fontweight': fweight}
                plt.title(str(jON)+'.'+j , **fontdict) 
            
                #if jN == 10:
                if jN == 5:
                    iN += 1                    
                    jN = 1
                else:
                    jN += 1
                
                if jON == 10:
                    jON = 1
                else:
                    jON += 1
                
                GENE_T["GENE"] = j
                RESULT = pd.concat([RESULT,GENE_T])
                    
                    
        #plt.rcParams["figure.autolayout"] = True
        #plt.tight_layout()
    
        #plt.savefig('./figures/'+Tit+'_'+sex+'_expression.png',dpi=500)
        plt.show()    
        
    return RESULT

In [ ]:
hub60_R10 = hub60[hub60['Rank'].isin(range(1,11))]
GENE=hub60_R10['gene']
RESULT = TCGA_plot(GENE,'GTEX_TCGA')